In [ ]:
import pandas as pd
import csv
import os,json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

#os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
!pip install biopython
import Bio
from Bio import SeqIO

     |████████████████████████████████| 2.3MB 8.3MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
os.listdir("/content/gdrive/My Drive/NPPI")
path="/content/gdrive/My Drive/NPPI"

In [ ]:
def JSLD(path,fn):
  with open(path+fn,"r") as f:
    D=json.load(f)
  return D

In [ ]:
SD={}
with open(path+"/FastaSeq/Protein.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        pid=(record.id).split("|")[1]
        seqn=str(record.seq)
        SD[pid]=seqn

print(len(SD))
      

2883


In [ ]:
def SeqEncoding(P2SQ):
    """  Encode Map   """
    FasD={"A":"#","G":"#","V":"#","C":"$","D":"&","E":"&","F":"@","I":"@","L":"@","P":"@",\
          "H":"*","N":"*","Q":"*","W":"*","K":"+","R":"+","M":"?","S":"?","T":"?","Y":"?"}
    NC=["B","J","O","U","X","Z"]## Removing 
    P2ESQ={}
    for pid in P2SQ:
        seq=P2SQ[pid]
        for xi in NC:
            if xi in seq:
                seq=seq.replace(xi,"")
        EnSeq=[str(FasD[x]) for x in seq]
        P2ESQ[pid]="".join(EnSeq)
    return P2ESQ

In [ ]:
TrS=SeqEncoding(SD)
print(len(TrS))

2883


In [ ]:
#P30504

In [ ]:
TrS['P30504']

'?+#?#@+#@@@@@?##@#@?&?*#$?*??+?@&?##?+@#+#&@+@@?##?#&&?*@#+@&?&##?@+#&@+#@*#&*&#@&?*&+&?*+?++*#*#&+#?@+*@+#??**?&&#?*?@*+??#$&@#@&#+@@+#?&*?#?&#+&?@#@*&&@+?*?##&?##*@?*++@&##+##&*@+#?@&#?$#&*@++?@&*#+&?@*+#&@@+?*#?**@@?&*&#?@+$*#@#@?@#&@?@?**+&#&&*?*&?&@#&?+@##&#?@*+*#####@?#*&*+??$*?**&#@*&@@?@?*&@??*@?@@@?#@###@##@##@##@####?#??$+++??##+##?$?*##$?*?#*#?&&?@@?$+#'

In [ ]:
global TrAA
def genRedSeqFeat():    
    A=["#","$","@","&","*","+","?"]
    TrAA=[]
    for i in A:
        for j in A:
            for k in A:
                TrAA.append(i+j+k)
    return TrAA
TrAA= genRedSeqFeat()  

def getFrq(seq):
    global TrAA; FrqMA=[]
    for fet in TrAA:
        FrqMA.append(seq.count(fet))
    return(FrqMA)
    
#cmpMinLen = lambda A, B :len([i for i in range(len(A)) if min(A[i],B[i])>0])

def MutualFreq(A,B):
    L=[]
    for i in range(len(A)):
        x=A[i];y=B[i];sc=0
        if (x and y):sc=-(min(x,y)/max(x,y))* np.log(min(x,y)/(x+y))
        L.append(sc)
    return L


In [ ]:
TrSD={}
for k in TrS:
  TrSD[k]=getFrq(TrS[k])
print(len(TrSD))
print(len(TrSD[k]))

2883
343


In [ ]:
with open(path+"/traindata/ProteinData/PData.json","w") as f:
  json.dump(TrSD,f)

In [ ]:
def dataGen(L):
  global TrSD
  Data=[];Label=[]
  
  for x in L:
    p1=x[0];p2=x[1];cl=int(x[2])
    d1= MutualFreq(TrSD[p1],TrSD[p2])
    Data.append(d1)
    Label.append(cl)
  Dx={'data':Data,'label':Label}
  return [Data,Label]
 

In [ ]:
def writeFile(fpath,fname, Dt):
  with open("{}{}.json".format(fpath,fname),"w") as f:
    json.dump(Dt,f)
  print("File write Complete")

In [ ]:
fnA="/traindata/ITR-1/SetA.json"
fnB="/traindata/ITR-1/SetB.json"
A=JSLD(path,fnA)
B=JSLD(path,fnB)
print(len(A), len(B))


26840 26840


In [ ]:
3355*5

16775

In [ ]:
"""
Ax=dataGen(A)  #  /traindata/ITR-1/SetA.json
writeFile(path+'/traindata/ITR-1/',"SetAD",Ax)
Bx=dataGen(B)
writeFile(path+'/traindata/ITR-1/',"SetBD",Bx)
"""

'\nAx=dataGen(A)  #  /traindata/ITR-1/SetA.json\nwriteFile(path+\'/traindata/ITR-1/\',"SetAD",Ax)\nBx=dataGen(B)\nwriteFile(path+\'/traindata/ITR-1/\',"SetBD",Bx)\n'

In [ ]:
from datetime import datetime
from joblib import dump, load
import os
from pathlib import Path
from sklearn import svm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, average_precision_score, recall_score, log_loss,\
accuracy_score,roc_auc_score,precision_recall_curve,f1_score,matthews_corrcoef

In [ ]:
classifiersX = {
  'SVM' : svm.SVC(C=2.0, kernel='rbf',gamma=0.0005, probability=True),#svm.SVC(kernel='linear', C=0.01),
  'RF' : RandomForestClassifier(n_estimators=192,min_samples_leaf = 75),  # random_forest
  #'decision_tree' : DecisionTreeClassifier(random_state=0),
  'KNN' : KNeighborsClassifier(n_neighbors=7),  # kneighbors
}

In [ ]:
def getFalsePos(predX,actualY,Ix):
  FPI=[]
  TP=FP=TN=FN=0
  for i,a in enumerate(actualY):
    if a==0:
      #print(i,a, predX[i])
    #if a==predX[i] and a==0:  it was  logically wrong for false positive(FP) computation. FP is those which are actually negative but predicted as positive
      
      if (predX[i]==1): # the corrected one
        FPI.append(i);FP+=1
      else:TN+=1

    elif (a==1):
      if (predX[i]==a): TP+=1
      else: FN+=1

  print("Total False Positive:fold-{}:: TP:{},FP:{},TN:{},FN:{}".format(Ix,TP,FP,TN,FN))
  return TP,FP,TN,FN


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
D=[[1,2],[3,4],[5,6],[7,8,9]]
XD= list(chunks(D, 2))
print(XD)


[[[1, 2], [3, 4]], [[5, 6], [7, 8, 9]]]


In [ ]:
def getPartition(D, L):
  p=sum(L); n=len(L)-p
  r=int(n/p)
  print (r, p, n)
  DL=[]; PS=[]; NS=[]; CL=[]
  for i,a in enumerate(L):
    if a==1: PS.append(D[i])
    else: NS.append(D[i])
  #print(len(NS))
  NL=list(chunks(NS,p))
  #print(len(NL))
  

  for k in NL:
    #print(len(k))
    
    XL=[]; YL=[]
    XL=PS+k
    #XL+=k
    YL=[1 for x in range (len(PS))]+[0 for x in range (len(k))]
    #YL+=[0 for x in range (len(k))]
    DL.append(np.array(XL)); CL.append(np.array(YL))
    #print (len(XL), len(YL), len(k), len(PS))
    #XL=[];YL=[]
    
  #print(len(DL), len(CL))
  return DL, CL

In [ ]:
def writeTFile1(fpath,S):
  f=open("{}".format(fpath),"a")
  f.write(S);f.close()
  

In [ ]:
def RunClassifier(Xi,Yi,Set, itr):
  dI=np.array(Xi[0])
  lI=np.array(Xi[1])
  testD=np.array(Yi[0]);testL=np.array(Yi[1])
  print(dI.shape,": " , lI.shape)
  print(testD.shape,": " , testL.shape)
  DL, CL = getPartition(Xi[0], Xi[1])
  PRd={}
  S=""
  for i,dataI in enumerate(DL): 
    labelI=CL[i]
    clf=classifiersX['RF']
    clf.fit(dataI, labelI)
    dump(clf, path+"/traindata/ITRi-{}/{}-Model.joblib".format(itr,Set))
    

    prediction = clf.predict(testD)
    TP,FP,TN,FN=getFalsePos(prediction,testL, i)
    PRd[i]=[[int(x) for x in (prediction)],Yi[1]]
    # PRd first index == fold

    probas_RF = clf.predict_proba(testD)

    AUC=roc_auc_score(testL,probas_RF[:, 1])
    ACCU=accuracy_score(testL,prediction)
    preRF, recRF, thrRF = precision_recall_curve(testL, probas_RF[:,1])
    AUPRC = auc(recRF, preRF)
    f1_s=f1_score(testL, prediction)
    mcc_s=matthews_corrcoef(testL, prediction)
    DScore={"AUC":AUC, "ACCU": ACCU, "AUPRC": AUPRC, "F1": f1_s, "MCC": mcc_s }

    S+="{},{},{},{},{},{},{},{},{},{},{},{}\n".format(itr,Set,i+1, AUC, ACCU,AUPRC,f1_s,mcc_s,TP,FP,TN,FN)
    
    #S="{}:AUC:{},ACCU:{},auPRC:{},F1:{},MCC;{}\n".format(i, AUC, ACCU,AUPRC,f1_s,mcc_s)
    # dump DScore as json
    #print(S)
    #########
    #rS="{},{},{}".format(itr,Set,S)
    
    #writeTFile1(path+'/traindata/allResult.csv',rS)
  
    #########


  with open(path+"/traindata/ITRi-{}/{}-predC.json".format(itr,Set),"w") as f:
    json.dump(PRd,f)
  return FP,S

In [ ]:
def writeTFile(fpath,fname,S):
  f=open("{}{}.txt".format(fpath,fname),"w")
  f.write(S);f.close()

In [ ]:
def UpdateItr(NS,B):
  UB=[]
  for i, a in enumerate(NS):
      UB.append(B[a])
  print("NewData Size= ", len(UB))
  return UB


In [ ]:
x=[1,1,1]
xyz=x[0]
for k in range(len(x)-1):
  xyz = xyz and x[k+1]
print(xyz)

1


In [ ]:
def loadFP(Set, itr):
  #Set="B"
  NewSet=[]

  count=0; iz=0
  with open(path+"/traindata/ITRi-{}/{}-predC.json".format(itr,Set),"r") as f:
    PRd= json.load(f)
  AC=PRd['0'][1]
  for k,a in enumerate(PRd['0'][0]):
    
    if (AC[k]==0):
      nz=PRd['0'][0][k]
      for ij in range(len(PRd)-1):
        nz = nz and PRd[str(ij+1)][0][k]
      #nz= PRd['0'][0][k] and PRd["1"][0][k] and PRd["2"][0][k] and PRd["3"][0][k] and PRd["4"][0][k] #number of false positives. all 1
      
      if nz==1:
        count+=1
      else:
        NewSet.append(k)
        iz+=1

    else:
      NewSet.append(k)

  print ("False Pos: {} ={}".format(Set,count))
  return NewSet
  
#updated negative set has become 16246
#pos still 3355 
#removed 529 values false pos
#NS=loadFP("B")

In [ ]:
def loadFPnew(Set, itr):
  #Set="B"
  NewSet=[]

  count=0; iz=0
  with open(path+"/traindata/ITRi-{}/{}-predC.json".format(itr,Set),"r") as f:
    PRd= json.load(f)
  AC=PRd['0'][1]
  for k,a in enumerate(PRd['0'][0]):
    
    if (AC[k]==0):
      nz=PRd['0'][0][k]
      nzi=nz
      for ij in range(len(PRd)-1):
        nz = nz and PRd[str(ij+1)][0][k]
        nzi+=PRd[str(ij+1)][0][k]
      #nz= PRd['0'][0][k] and PRd["1"][0][k] and PRd["2"][0][k] and PRd["3"][0][k] and PRd["4"][0][k] #number of false positives. all 1
      
      if nzi>2:
        count+=1
      else:
        NewSet.append(k)
        iz+=1

    else:
      NewSet.append(k)

  print ("False Pos: {} ={}".format(Set,count))
  return NewSet
  
#updated negative set has become 16246
#pos still 3355 
#removed 529 values false pos
#NS=loadFP("B")

In [ ]:
def Process(A,B,itr):
  Ax=dataGen(A)  #  /traindata/ITR-1/SetA.json
  writeFile(path+'/traindata/ITRi-{}/'.format(itr),"SetAD",Ax)
  Bx=dataGen(B)
  writeFile(path+'/traindata/ITRi-{}/'.format(itr),"SetBD",Bx)
  FPb,SB=RunClassifier(Ax,Bx,"B",itr)
  FPa,SA=RunClassifier(Bx,Ax,"A",itr)

  #writeTFile(path+'/traindata/ITR-{}/'.format(itr),"ResSetB",SB)

  #writeTFile(path+'/traindata/ITR-{}/'.format(itr),"ResSetA",SA)
  
  #rSB="{},{},{}".format(itr,"B",SB)
  #rSA="{},{},{}".format(itr,"A",SA)

  writeTFile1(path+'/traindata/allResult.csv',SB)
  
  writeTFile1(path+'/traindata/allResult.csv',SA)
 
  print("B")
  NS=loadFPnew("B", itr)
  nB=UpdateItr(NS,B)

  writeFile(path+'/traindata/ITRi-{}/'.format(itr+1),"SetB",nB)
  print("A")
  NS=loadFPnew("A", itr)
  nA=UpdateItr(NS,A)

  writeFile(path+'/traindata/ITRi-{}/'.format(itr+1),"SetA",nA)
  return nA, nB

In [ ]:
fnA="/traindata/ITR-1/SetA.json"
fnB="/traindata/ITR-1/SetB.json"
A=JSLD(path,fnA)
B=JSLD(path,fnB)
#print(len(A), len(B))

In [ ]:
for itr in range(1,3):
  print(itr)

1
2


In [ ]:
S="itr,Set,Fold, AUC, ACCU,AUPRC,f1_s,mcc_s,TP,FP,TN,FN \n"
#writeTFile(path+'/traindata/allResult.csv',rSA)
f=open(path+'/traindata/allResult.csv',"w")
f.write(S);f.close()

for itr in range(1,20):
  resDir=path+"/traindata/ITRi-{}".format(itr)
  resDir1=path+"/traindata/ITRi-{}".format(itr+1)
  if not os.path.exists(resDir):
    os.makedirs(resDir)
  if not os.path.exists(resDir1):
    os.makedirs(resDir1)
  print ("A= {}, B={}".format(len(A),len(B)))
  A,B=Process(A,B,itr)
  print("----------Iteration : {}-------------".format(itr))

A= 26840, B=26840
File write Complete
File write Complete
(26840, 343) :  (26840,)
(26840, 343) :  (26840,)
7 3355 23485
Total False Positive:fold-0:: TP:2412,FP:2974,TN:20511,FN:943
Total False Positive:fold-1:: TP:2378,FP:2591,TN:20894,FN:977
Total False Positive:fold-2:: TP:2431,FP:2985,TN:20500,FN:924
Total False Positive:fold-3:: TP:2395,FP:2225,TN:21260,FN:960
Total False Positive:fold-4:: TP:2418,FP:2416,TN:21069,FN:937
Total False Positive:fold-5:: TP:2425,FP:2913,TN:20572,FN:930
Total False Positive:fold-6:: TP:2352,FP:2618,TN:20867,FN:1003
(26840, 343) :  (26840,)
(26840, 343) :  (26840,)
7 3355 23485
Total False Positive:fold-0:: TP:2395,FP:2323,TN:21162,FN:960
Total False Positive:fold-1:: TP:2335,FP:2612,TN:20873,FN:1020
Total False Positive:fold-2:: TP:2420,FP:2503,TN:20982,FN:935
Total False Positive:fold-3:: TP:2364,FP:2528,TN:20957,FN:991
Total False Positive:fold-4:: TP:2360,FP:2859,TN:20626,FN:995
Total False Positive:fold-5:: TP:2390,FP:2975,TN:20510,FN:965
Total Fa

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


(23868, 343) :  (23868,)
(23886, 343) :  (23886,)
6 3355 20513
Total False Positive:fold-0:: TP:2393,FP:1704,TN:18827,FN:962
Total False Positive:fold-1:: TP:2380,FP:1828,TN:18703,FN:975
Total False Positive:fold-2:: TP:2427,FP:1951,TN:18580,FN:928
Total False Positive:fold-3:: TP:2402,FP:1819,TN:18712,FN:953
Total False Positive:fold-4:: TP:2426,FP:2002,TN:18529,FN:929
Total False Positive:fold-5:: TP:2476,FP:2030,TN:18501,FN:879
Total False Positive:fold-6:: TP:3355,FP:20531,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


B
False Pos: B =2506
NewData Size=  21362
File write Complete
A
False Pos: A =2651
NewData Size=  21235
File write Complete
----------Iteration : 2-------------
A= 21235, B=21362
File write Complete
File write Complete
(21235, 343) :  (21235,)
(21362, 343) :  (21362,)
5 3355 17880
Total False Positive:fold-0:: TP:2463,FP:1055,TN:16952,FN:892
Total False Positive:fold-1:: TP:2486,FP:1087,TN:16920,FN:869
Total False Positive:fold-2:: TP:2481,FP:939,TN:17068,FN:874
Total False Positive:fold-3:: TP:2464,FP:950,TN:17057,FN:891
Total False Positive:fold-4:: TP:2459,FP:937,TN:17070,FN:896
Total False Positive:fold-5:: TP:3355,FP:18007,TN:0,FN:0


...

(16939, 343) :  (16939,)
(16999, 343) :  (16999,)
4 3355 13584
Total False Positive:fold-0:: TP:2564,FP:277,TN:13367,FN:791
Total False Positive:fold-1:: TP:2570,FP:254,TN:13390,FN:785
Total False Positive:fold-2:: TP:2552,FP:247,TN:13397,FN:803
Total False Positive:fold-3:: TP:2555,FP:165,TN:13479,FN:800
Total False Positive:fold-4:: TP:3355,FP:13644,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


B
False Pos: B =193
NewData Size=  16746
File write Complete
A
False Pos: A =223
NewData Size=  16776
File write Complete
----------Iteration : 10-------------
A= 16776, B=16746
File write Complete
File write Complete
(16776, 343) :  (16776,)
(16746, 343) :  (16746,)
4 3355 13421
Total False Positive:fold-0:: TP:2559,FP:251,TN:13140,FN:796
Total False Positive:fold-1:: TP:2554,FP:179,TN:13212,FN:801
Total False Positive:fold-2:: TP:2538,FP:126,TN:13265,FN:817
Total False Positive:fold-3:: TP:2567,FP:205,TN:13186,FN:788
Total False Positive:fold-4:: TP:3355,FP:13391,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


(16746, 343) :  (16746,)
(16776, 343) :  (16776,)
3 3355 13391
Total False Positive:fold-0:: TP:2570,FP:261,TN:13160,FN:785
Total False Positive:fold-1:: TP:2553,FP:174,TN:13247,FN:802
Total False Positive:fold-2:: TP:2536,FP:254,TN:13167,FN:819
Total False Positive:fold-3:: TP:2593,FP:192,TN:13229,FN:762
B
False Pos: B =168
NewData Size=  16578
File write Complete
A
False Pos: A =54
NewData Size=  16722
File write Complete
----------Iteration : 11-------------
A= 16722, B=16578
File write Complete
File write Complete
(16722, 343) :  (16722,)
(16578, 343) :  (16578,)
3 3355 13367
Total False Positive:fold-0:: TP:2589,FP:184,TN:13039,FN:766
Total False Positive:fold-1:: TP:2549,FP:121,TN:13102,FN:806
Total False Positive:fold-2:: TP:2542,FP:117,TN:13106,FN:813
Total False Positive:fold-3:: TP:2604,FP:159,TN:13064,FN:751
(16578, 343) :  (16578,)
(16722, 343) :  (16722,)
3 3355 13223
Total False Positive:fold-0:: TP:2561,FP:255,TN:13112,FN:794
Total False Positive:fold-1:: TP:2540,FP:255,